In [1]:
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel

# slightly modified version of CLIP_Implementation.ipynb that writes out the relevant information to a file
# that we can use for later analysis

auth_token = "" # use your own
winoground = load_dataset("facebook/winoground", use_auth_token=auth_token)["test"]

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

f = open("CLIP_scores.txt", "w")
f.write("ID\xa0tag\xa0secondary_tag\xa0num_main_preds\xa0collapsed_tag\xa0C0I0\xa0C0I1\xa0C1I0\xa0C1I1\n")
print(winoground[4])

/home/samuelyu/.conda/envs/CLEAR/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 2/2 [00:03<00:00,  1.60s/it]


Dataset winoground downloaded and prepared to /home/samuelyu/.cache/huggingface/datasets/facebook___winoground/default/0.0.0/ce486f3e39fab90997d6f3c58c4b0103eb9c37011049ef775a465f0ab2e78d7d. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 200.62it/s]
Downloading: 100%|██████████| 4.19k/4.19k [00:00<00:00, 2.16MB/s]
Downloading: 100%|██████████| 605M/605M [00:05<00:00, 104MB/s]  
Downloading: 100%|██████████| 568/568 [00:00<00:00, 338kB/s]
Downloading: 100%|██████████| 862k/862k [00:00<00:00, 10.6MB/s]
Downloading: 100%|██████████| 525k/525k [00:00<00:00, 7.35MB/s]
Downloading: 100%|██████████| 2.22M/2.22M [00:00<00:00, 20.1MB/s]
Downloading: 100%|██████████| 389/389 [00:00<00:00, 202kB/s]


{'id': 4, 'image_0': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1280x1920 at 0x7FDE17F5F880>, 'image_1': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1431 at 0x7FDE17F5FC40>, 'caption_0': 'the person without earrings pays the person with earrings', 'caption_1': 'the person with earrings pays the person without earrings', 'tag': 'Negation, Scope', 'secondary_tag': 'Morpheme-Level', 'num_main_preds': 1, 'collapsed_tag': 'Relation'}


In [ ]:
import clip

In [ ]:
model, preprocess = clip.load("ViT-B/32")

100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 71.2MiB/s]


In [ ]:
model = model.float().cuda()

In [ ]:
preprocess(winoground[0]['image_0']).shape

torch.Size([3, 224, 224])

In [ ]:
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters())

In [ ]:

for i in range(1000):
    optimizer.zero_grad()
    train_data = winoground[np.random.choice(400, size=32, replace=False)]
    train_images = [train_data['image_0'] + train_data['image_1']]
    train_images = [preprocess(img) for img in train_images[0]]
    train_images = torch.stack(train_images, dim=0).cuda()
    train_text = clip.tokenize(train_data['caption_0'] + train_data['caption_1']).cuda()
#     # train_images = [preprocess(j['image_0']) for j in train_data] + [preprocess(j['image_1']) for j in train_data]
#     train_images = torch.stack(train_images, dim=0)
        
#     train_text = [j['caption_0'] for j in train_data] + [(j['caption_1']) for j in train_data]
    # train_text = clip.tokenize(train_text).cuda()
    
    logits_per_image, logits_per_text = model(train_images, train_text)
    
    loss_i = loss(logits_per_image, torch.arange(64).cuda())
    loss_t = loss(logits_per_text, torch.arange(64).cuda())
    
    total_loss = loss_i + loss_t
    total_loss.backward()
    print(total_loss)


tensor(3.8613, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1771, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.5796, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6946, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7980, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0835, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2928, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0485, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1439, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1366, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1870, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9723, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.5228, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.6926, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7157, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2331, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.0165, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1654, device='cuda:0', grad_fn=<AddBack

NameError: name 'j' is not defined